# Bayesian Hierarchical Regression

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm

from pandas_datareader import DataReader

%matplotlib inline

## Hierarchical Models: Cafes

The following example is based on material from Richard McElreath and the "Statistical Rethinking" book.

Suppose you were tasked with providing the mean and standard deviation for the waiting time to get a cup of coffee for every cafe in the world.

How would one go about doing this?

#### Method 1: Pooled

One could treat each cafe the same and use visits to many different cafes to estimate a world-wide mean and standard deviation of waiting times for cafes.

This method disregards the fact that cafes have any differences -- Even if all cafes in the world were run by Starbucks, there would be differences in waiting times due to factors like foot traffic and staff.

#### Method 2: Amnesia

If one was able to visit every cafe in the world repeatedly, one could estimate a cafe-specific mean and standard deviation.

We call this method the "amnesia method" because it is like you are forgetting that you've ever visited any other cafe in the world... This seems suboptimal because repeated visits to every cafe in the world would become infeasible and, although cafes differ, they also have similarities (which this method ignores).

#### Method 3: Multilevel (Hierarchical)

Rather than treat cafes as completely the same (pooled) or completely separate (amnesia), multilevel models recognize that there are similarities and differences and these models attempt to separate these similarities and differences.

## CAPM

We aren't going to discuss in detail about the interpretation (or motivation) of the CAPM model today (stay tuned for more on this...), but we think it is a compelling example for hierarchical models.

The CAPM is described by the following equation:

$$r_{i, t} = r_{f, t} + \beta_i (r_{m, t} - r_{f, t})$$

There are a few components to this equation:

* $r_{f, t}$: The risk-free return <-- We will use the 3 month treasury bills to approximate this
* $r_{m, t}$: The market return <-- We will use the S&P 500 to approximate the market return
* $r_{i, t}$: The return of a particular asset
* $\beta_i$: The beta of the asset <-- This is a measure of relative co-movement with the market returns

## Data Ingestion

We will download data from three sources:

1. [Wikipedia's list of S&P 500 companies](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies)
2. [Yahoo Finance](https://finance.yahoo.com/)
3. [Federal Reserve Bank of St Louis](https://fred.stlouisfed.org/)

Yahoo Finance previously had an API that one could use to download data, but they deprecated this API in 2017...

In response to this, open source contributors have created a new Python package called `yfinance` which you can read about [here](https://github.com/ranaroussi/yfinance). We will use this package to download daily stock prices -- If you'd like to replicate this data gathering, you'll need to run `pip install yfinance` to install the `yfinance` package.

### S&P 500 Metadata

There are several pieces of metadata that we are particularly interested in here:

* `ticker`: This will tell us the ticker that we can use to download the data
* `gics`: The [GICS sector](https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard) tells us broad information about the industry that a company works in. An example of a few of these industries would be: _energy_, _materials_, _consumer discretionary_, _financials_, etc...
* `gics_subindustry`: The GICS subcategory tells us more specific information about the industry that a company works in. For example, _asset management and custody_, _oil and gas drilling_, _textiles_, _automobile manufacturers_, etc...
* `hq_location`: The location of the company headquarters
* `start_date`: When the company entered the S&P 500
* `founded`: The year the company was founded

In [ ]:
# Read in tables from wikipedia page
sp500_tables = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
)

# Get table of companies and rename
sp500 = sp500_tables[0].rename(columns={
    "Symbol": "ticker",
    "GICS Sector": "gics",
    "GICS Sub-Industry": "gics_subindustry",
    "Headquarters Location": "hq_location",
    "Date first added": "start_date",
})
sp500.columns = [c.lower() for c in sp500.columns]

# Yahoo uses - rather than .
sp500["ticker"] = sp500["ticker"].str.replace(".", "-")

# Add Zoom for fun
sp500 = sp500.append(
    {
        "ticker": "ZM",
        "security": "Zoom",
        "sec filings": "reports",
        "gics": "Communication Services",
        "gics_subindustry": "Telecom Services",
        "hq_location": "San Jose, CA",
        "start_date": "2019-03-22",
        "cik": 1585521,
        "founded": "2011",
    }, ignore_index=True
)

# Extract tickers
sp500_tickers = sp500["ticker"].to_list()

sp500.to_parquet("../../Data/sp500/sp500_companies.parquet")

In [ ]:
sp500.head(10)

### S&P Prices

The other piece of data is the actual stock prices. We are given a few options for which type of price data to report:

* `Close`: The last price that the stock traded at on a particular day
* `Open`: The first price that the stock traded at on a particular day
* `High`: The highest price that a stock traded at on a particular day
* `Low`: The lowest price that a stock traded at on a particular day
* `Adj Close`: The close price adjusted for dividends, splits, and other corporate actions <-- We will be using adjusted close prices

In [ ]:
import yfinance as yf  # Make sure to install if not installed!

raw_prices = yf.download(
    tickers=sp500_tickers + ["^GSPC"],
    start="2015-12-31", end="2021-01-01",
)

In [ ]:
prices = raw_prices.loc[:, "Close"].reset_index().rename(
    columns={
        "Date": "dt"
    }
).melt(
    id_vars="dt",
    var_name="ticker",
    value_name="price"
).replace(
    {"ticker": {"^GSPC": "sp500"}}
)

prices.to_parquet("../../Data/sp500/prices.parquet")

In [ ]:
prices.head()

### Treasury Bills

We will use the (annualized) return on US treasury bills as a baseline "risk-free" return.

A treasury bill is a form of government debt in which you pay the government a price $p_{\tau}$ at time $t$ and are repaid \\$1,000 at time $t + \tau$ (with time measured in months).

The implied return is then $r = \times \frac{1000 - p}{p}$ which can then be annualized based on the maturity $\tau$ by 

$$r_{\text{annualized}} = 1 - \left(1 + \frac{1000 -  p}{p} \right)^{\frac{12}{\tau}}$$

We will report the rates in annual percent return (i.e. $r_\text{annualized} = 0.01$ will be reported as 1\%).

In [ ]:
tbills = DataReader(
    "DTB3", "fred", start=2014, end=2021
).reset_index().rename(
    columns={"DTB3": "riskfree", "DATE": "dt"}
).groupby(
    pd.Grouper(key="dt", freq="M")
).mean()

# Convert to monthly
tbills["riskfree"] = tbills.eval(
    "100 * ((1 + riskfree/100)**(1/12) - 1)"
)

tbills.reset_index().to_parquet("../../Data/sp500/tbills.parquet")


In [ ]:
tbills.tail()

### Merging metadata and prices

Eventually we will want to "group" certain stocks together based on characteristics from their metadata.

We allow for this by merging the metadata and price data:

In [ ]:
df = prices.merge(
    sp500.loc[:, ["ticker", "gics", "gics_subindustry"]],
    on="ticker", how="left"
).sort_values(["ticker", "dt"])

df.head()

### Computing returns and merging T-bills

We will need to compute returns at various points in time.

We define a function to help us do this:

In [ ]:
def compute_returns(df_in, freq="M"):
    gb = df_in.groupby(
        ["ticker", pd.Grouper(key="dt", freq=freq)]
    )

    # Get start and end prices
    sp500_ms = gb["price"].first()
    sp500_me = gb["price"].last()

    # Return last value of every other column
    out = pd.DataFrame(
        100 * (sp500_me - sp500_ms) / sp500_ms
    ).rename(
        columns={"price": "returns"}
    ).reset_index()

    cols_to_iterate = [
        c for c in df_in.columns
        if c not in ["dt", "ticker", "price"]
    ]
    for c in cols_to_iterate:
        out.loc[:, c] = gb[c].last().values

    return out


In [ ]:
returns = compute_returns(df)

component_returns = returns.query("ticker != 'sp500'")
index_returns = returns.query(
    "ticker == 'sp500'"
).loc[:, ["dt", "returns"]].rename(
    columns={"returns": "market"}
)

In [ ]:
component_returns.tail()

In [ ]:
returns = component_returns.merge(
    tbills, on="dt", how="left"
).merge(
    index_returns, on="dt", how="left"
).dropna()

In [ ]:
returns.head()

## A Common Approach to CAPM:

A common approach to CAPM is to specify a completely separate regression for each stock.

We will build a Bayesian version of this regression (to keep the baseline methodology similar), but this regression could also be done (more quickly) using least squares...

Our model will be described by

\begin{align*}
   r_{i, t} - r_{f, t} &= \beta_i (r_{m, t} - r_{f, t}) + \sigma_i \varepsilon_{i, t}\\
   \sigma_i &\sim \text{HalfStudentT}(4) \\
   \beta_i &\sim N(0, 5)
\end{align*}

### Single stock

We begin by just computing the $\beta_{\text{ZM}}$ and $\sigma_{\text{ZM}}$ for Zoom

In [ ]:
m_zoom = pm.Model()

# Basic data
zm_returns = returns.query("ticker == 'ZM'")
ri_m_rf = zm_returns.eval("returns - riskfree").to_numpy()
rm_m_rf = zm_returns.eval("market - riskfree").to_numpy()

with m_zoom:
    # Data
    _ri_m_rf = pm.Data("ri_m_rf", ri_m_rf)
    _rm_m_rf = pm.Data("rm_m_rf", rm_m_rf)

    # Prior
    beta_i = pm.Normal("beta_i", 0.0, 5.0)
    sigma_i = pm.HalfStudentT("sigma_i", 2)

    # Likelihood
    ll = pm.Normal("ll", beta_i*_rm_m_rf, sigma_i, observed=_ri_m_rf)

In [ ]:
with m_zoom:
    traces_zoom = pm.sample(3500, tune=1000)

#### Posterior

In [ ]:
with m_zoom:
    az.plot_trace(traces_zoom)

### Many stocks

`pymc3` also allows us to estimate many regressions at once (there is no interdependence in this regression)

In [ ]:
returns.head()

In [ ]:
m_baseline = pm.Model()

# Basic data
tickers = returns["ticker"].unique()
ntickers = tickers.shape[0]
ticker_2_int = dict(zip(tickers, range(ntickers)))
int_2_ticker = {v: k for k, v in ticker_2_int.items()}
ri_m_rf = returns.eval("returns - riskfree").to_numpy()
rm_m_rf = returns.eval("market - riskfree").to_numpy()
ticker_idx = returns["ticker"].map(lambda x: ticker_2_int[x]).to_numpy()

with m_baseline:
    # Data
    _ri_m_rf = pm.Data("ri_m_rf", ri_m_rf)
    _rm_m_rf = pm.Data("rm_m_rf", rm_m_rf)
    _ticker_idx = pm.intX(pm.Data("ticker_idx", ticker_idx))

    # Prior
    beta_i = pm.Normal("beta_i", 0.0, 5.0, shape=ntickers)
    sigma_i = pm.HalfStudentT("sigma_i", 5.0, shape=ntickers)

    # Likelihood
    ll = pm.Normal("ll", beta_i[_ticker_idx]*_rm_m_rf, sigma_i[_ticker_idx], observed=_ri_m_rf)

In [ ]:
with m_baseline:
    traces_baseline = pm.sample(2500, tune=1000)

#### Posterior of "amnesia" $\beta$s

In [ ]:
betas_nopool = pd.DataFrame(
    {
        "beta_mean": traces_baseline["beta_i"].mean(axis=0),
        "beta_std": traces_baseline["beta_i"].std(axis=0),
        "sigma_mean": traces_baseline["sigma_i"].mean(axis=0),
        "sigma_std": traces_baseline["sigma_i"].std(axis=0)
    }, index=tickers
)

betas_nopool.T.head()

**Plausbility?**

Recall that in our single stock estimation, we found $\beta_{\text{Zoom}} < 0$...

Do many other stocks have negative $\beta$? How plausible is this given the rest of our estimates? 

In [ ]:
betas_nopool.query("beta_mean < 0.0")

### A Hierarchical Approach to CAPM

We now turn to building a hierarchical version of this model.

The purpose of the hierarchical model is to allow groups of observations to learn from one another -- We have, at most, 60 observations for each of our stocks because we are computing the 5 year beta (similar to what Yahoo Finance reports).


#### Grouping observations

As we've mentioned previously, model-building is an art that takes practice (and watching others do it)...

Hierarchical models are no exception and choosing how to group your observations take practice (and experimentation).

**Hyperparameters and hyperpriors**

A _hyperparameter_ is a parameter that's an input to a prior. For example, in our previous example we specified $\beta_i \sim N(0, 5)$ so 0 and 5 were hyperparameters.

A _hyperprior_ is a prior on a hyperparameter

(Any guess what a hyperhyperprior is? It is a prior on the hyperhyperparameters which are the parameters governing the hyperprior...)

**Hierarchical model**

Hyperpriors will be a central feature of hierarchical models and they will be used to group observations. In our example, we originally wrote the following model

\begin{align*}
   r_{i, t} - r_{f, t} &= \beta_i (r_{m, t} - r_{f, t}) + \sigma_i \varepsilon_{i, t}\\
   \beta_i &\sim N(0, 5) \\
   \sigma_i &\sim \text{HalfStudentT}(4)
\end{align*}

A hierarchical version of the model might be specified as

\begin{align*}
   r_{i, t} - r_{f, t} &= \beta_i (r_{m, t} - r_{f, t}) + \sigma_i \varepsilon_{i, t}\\
   \sigma_i &\sim \text{HalfStudentT}(2) \\
   \beta_i &\sim N(\hat{\mu}_j, \hat{\sigma}_j) \\
   \hat{\mu}_j &\sim N(0, 5) \\
   \hat{\sigma}_j &\sim \text{HalfStudentT}(4)
\end{align*}

where $j$ could indicate the GICS sector (`gics`) that $i$ is identified by.

#### Writing down the model in pymc3

\begin{align*}
   r_{i, t} - r_{f, t} &= \beta_i (r_{m, t} - r_{f, t}) + \sigma_i \varepsilon_{i, t}\\
   \sigma_i &\sim \text{HalfStudentT}(2) \\
   \beta_i &\sim N(\hat{\mu}_j, \hat{\sigma}_j) \\
   \hat{\mu}_j &\sim N(0, 5) \\
   \hat{\sigma}_j &\sim \text{HalfStudentT}(2)
\end{align*}

In [ ]:
# Ticker/subindustry
_tick_sect = returns.loc[:, ["ticker", "gics"]]
tick_sect = _tick_sect.loc[~_tick_sect.duplicated(keep="first"), :]
tickers = tick_sect["ticker"].to_numpy()
ntickers = tickers.shape[0]
sects = tick_sect["gics"].unique()
nsect = sects.shape[0]

# Mappings
ticker_2_int = dict(zip(tickers, range(ntickers)))
int_2_ticker = {v: k for k, v in ticker_2_int.items()}  # Only reverse when unique
sect_2_int = dict(zip(sects, range(nsect)))
int_2_sect = {v: k for k, v in sect_2_int.items()}  # Only reverse when unique
ticker_2_sect = dict(
    zip(
        tick_sect["ticker"].map(ticker_2_int).to_numpy(),
        tick_sect["gics"].map(sect_2_int).to_numpy()
    )
)

# Data
ri_m_rf = returns.eval("returns - riskfree").to_numpy()
rm_m_rf = returns.eval("market - riskfree").to_numpy()
ticker_idx = returns["ticker"].map(lambda x: ticker_2_int[x]).to_numpy()
sect_idx = np.array([ticker_2_sect[x] for x in range(ntickers)])

In [ ]:
m_hierarchical = pm.Model()

with m_hierarchical:
    # Data
    _ri_m_rf = pm.Data("ri_m_rf", ri_m_rf)
    _rm_m_rf = pm.Data("rm_m_rf", rm_m_rf)
    _ticker_idx = pm.intX(pm.Data("ticker_idx", ticker_idx))
    _sect_idx = pm.intX(pm.Data("sect_idx", sect_idx))

    # Hyperprior
    muhat = pm.Normal("mu_hat", 0, 5, shape=nsect)
    sigmahat = pm.HalfStudentT("sigma_hat", 5, shape=nsect)

    # Prior
    beta_i = pm.Normal(
        "beta_i", muhat[_sect_idx], sigmahat[_sect_idx],
        shape=ntickers
    )
    sigma_i = pm.HalfStudentT("sigma_i", 5, shape=ntickers)

    # Likelihood
    ll = pm.Normal("ll", beta_i[_ticker_idx]*_rm_m_rf, sigma_i[_ticker_idx], observed=_ri_m_rf)

In [ ]:
with m_hierarchical:
    traces_hierarchical = pm.sample(2000, tune=1500)

**Diagnosing MCMC chains**

There are a few diagnostic tools available:

1. Rhat: This is a measure of how different the posterior samples across different chains are.
  - You would like rhat to be very close to 1 -- If it's much higher than 1.01 or 1.02 then you should be worried about whether you're drawing samples from your posterior
2. ESS: The effective sample size. If your draws are highly correlated (or have other problems) then the ESS becomes smaller
  - It is possible to draw 1,000 values from your posterior but have less than 100 effective samples.
  - Can compute the relative ESS which is $\text{ESS} / n$

In [ ]:
with m_hierarchical:
    ess = az.stats.diagnostics.ess(traces_hierarchical, relative=True)
    rhat = az.stats.diagnostics.rhat(traces_hierarchical)

In [ ]:
for _var in ["beta_i", "sigma_i", "mu_hat", "sigma_hat"]:
    print(f"Variable: {_var}")
    print("\tMinimum relative ESS: ", ess[_var].min().values)
    print("\tMaximum rhat: ", rhat[_var].max().values)

In [ ]:
traces_hierarchical.report._chain_warnings

**Zoom's $\beta$**

Note that Zoom is an element of the "Communication Services" sector

In [ ]:
tick_sect.query("ticker == 'ZM'")

Which is an identified by

In [ ]:
sect_2_int["Communication Services"]

In [ ]:
with m_hierarchical:
    az.plot_trace(traces_hierarchical, var_names="mu]_hat")

In [ ]:
with m_hierarchical:
    az.plot_trace(traces_hierarchical, var_names="sigma_hat")

In [ ]:
betas_nopool

In [ ]:
betas_hierarchical = pd.DataFrame(
    {
        "beta_mean": traces_hierarchical["beta_i"].mean(axis=0),
        "beta_std": traces_hierarchical["beta_i"].std(axis=0),
        "sigma_mean": traces_hierarchical["sigma_i"].mean(axis=0),
        "sigma_std": traces_hierarchical["sigma_i"].std(axis=0)
    }, index=tickers
)

betas_hierarchical

In [ ]:
tick_sect

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))

sector = "Communication Services"
foo_tickers = tick_sect.query(f"gics == '{sector}'").loc[:, "ticker"].to_numpy()
foo = betas_nopool.loc[foo_tickers, :]
bar = betas_hierarchical.loc[foo_tickers, :]
nfoo = foo.shape[0]

ax.scatter(np.arange(nfoo), foo["beta_mean"], color="r", s=15)
ax.scatter(np.arange(nfoo), bar["beta_mean"], color="b", s=15)

## Conclusion

We conclude with this excerpt about the benefits of multilevel (hierarchical) modeling from Statistical Rethinking by Richard McElreath:

> 1. Improved estimates for repeat sampling. When more than one observation arises from the same individual, location, or time, then traditional, single-level models either maximally underfit or overfit the data.
> 2. Improved estimates for imbalance in sampling. When some individuals, locations, or times are sampled more than others, multilevel models automatically cope with differing uncertainty across these clusters. This prevents over-sampled clusters from unfairly dominating inference.
> 3. Estimates of variation. If our research questions include variation among individuals or other groups within the data, then multilevel models are a big help, because they model variation explicitly.
> 4. Avoid averaging, retain variation. Frequently, scholars pre-average some data to construct variables. This can be dangerous, because averaging removes variation, and there are also typically several different ways to perform the averaging. Averaging therefore manufactures false confidence and introduces arbitrary data transformations. Multilevel models allow us to preserve the uncertainty and avoid data transformations